In [6]:
import requests
import pandas as pd
import mysql.connector as db
import numpy as np
from datetime import datetime


In [28]:
import pymysql

mydb = pymysql.connect(
    host='localhost',
    user='root',
    password='Ke$hw0rd-12345',
    database='gitdata'
)
mycursor = mydb.cursor()

In [8]:
# Topics to fetch data for
topics = ["Gen AI","Powerbi", "deep learning", "MLOps",
          "Quantum Computing", "python",
          "machine learning","robotics"]

In [9]:
def fetch_github_data(topic):
    url = f"https://api.github.com/search/repositories?q={topic}&sort=stars"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for {topic}")
        return None

# Extract data for all topics
data_list = []
for topic in topics:
    data = fetch_github_data(topic)
    if data:
        for item in data['items']:
            data_list.append({
                'Repository_Name': item['name'],
                'Owner': item['owner']['login'],
                'Description': item['description'],
                'URL': item['html_url'],
                'Programming_Language': item.get('language', 'Unknown'),
                'Creation_Date': item['created_at'],
                'Last_Updated_Date': item['updated_at'],
                'Number_of_Stars': item['stargazers_count'],
                'Number_of_Forks': item['forks_count'],
                'Number_of_Open_Issues': item['open_issues_count'],
                'License_Type': item['license']['name'] if item['license'] else 'No License'
            })

df = pd.DataFrame(data_list)
df.to_csv('github_repositor.csv', index=False)
print(df)

                 Repository_Name                Owner  \
0    generative-ai-for-beginners            microsoft   
1              generative-models         Stability-AI   
2                        danswer           danswer-ai   
3    awesome-generative-ai-guide          aishwaryanr   
4                  generative-ai  GoogleCloudPlatform   
..                           ...                  ...   
235        awesome-multimodal-ml            pliang279   
236              node-serialport           serialport   
237                 ai-deadlines       paperswithcode   
238                    OpenMower       ClemensElflein   
239             awesome-robotics             kiloreux   

                                           Description  \
0    21 Lessons, Get Started Building with Generati...   
1                    Generative Models by Stability AI   
2    Gen-AI Chat for Teams - Think ChatGPT if it ha...   
3    A one stop repository for generative AI resear...   
4    Sample code and note

In [12]:
# Data Cleaning
df.shape

(240, 11)

In [14]:
df.info() # show full info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Repository_Name        240 non-null    object
 1   Owner                  240 non-null    object
 2   Description            235 non-null    object
 3   URL                    240 non-null    object
 4   Programming_Language   202 non-null    object
 5   Creation_Date          240 non-null    object
 6   Last_Updated_Date      240 non-null    object
 7   Number_of_Stars        240 non-null    int64 
 8   Number_of_Forks        240 non-null    int64 
 9   Number_of_Open_Issues  240 non-null    int64 
 10  License_Type           240 non-null    object
dtypes: int64(3), object(8)
memory usage: 20.8+ KB


In [18]:
df.isna().sum() # check null

Repository_Name           0
Owner                     0
Description               0
URL                       0
Programming_Language     38
Creation_Date             0
Last_Updated_Date         0
Number_of_Stars           0
Number_of_Forks           0
Number_of_Open_Issues     0
License_Type              0
dtype: int64

In [17]:
df['Creation_Date'] = pd.to_datetime(df['Creation_Date']) # convert data type
df['Last_Updated_Date'] = pd.to_datetime(df['Last_Updated_Date'])

# HANDLING NULL VALUES

df = df.replace({np.nan: None})
df['Description'].fillna('No Description', inplace=True)
df['License_Type'].fillna('No License', inplace=True)

# Preview cleaned data
print(df.head())

               Repository_Name                Owner  \
0  generative-ai-for-beginners            microsoft   
1            generative-models         Stability-AI   
2                      danswer           danswer-ai   
3  awesome-generative-ai-guide          aishwaryanr   
4                generative-ai  GoogleCloudPlatform   

                                         Description  \
0  21 Lessons, Get Started Building with Generati...   
1                  Generative Models by Stability AI   
2  Gen-AI Chat for Teams - Think ChatGPT if it ha...   
3  A one stop repository for generative AI resear...   
4  Sample code and notebooks for Generative AI on...   

                                                 URL Programming_Language  \
0  https://github.com/microsoft/generative-ai-for...     Jupyter Notebook   
1  https://github.com/Stability-AI/generative-models               Python   
2              https://github.com/danswer-ai/danswer               Python   
3  https://github.com/ai

C:\Users\kkeshav\AppData\Local\Temp\ipykernel_3888\1911430551.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Description'].fillna('No Description', inplace=True)
C:\Users\kkeshav\AppData\Local\Temp\ipykernel_3888\1911430551.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [20]:
df = pd.read_csv("github_repositor.csv")

# Convert NaN to None 
df = df.replace({np.nan: None})

df['Creation_Date'] = pd.to_datetime(df['Creation_Date']).dt.date
df['Last_Updated_Date'] = pd.to_datetime(df['Last_Updated_Date']).dt.date

In [29]:
# Create the repositories table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS reposit (
    Repository_Name VARCHAR(255) NOT NULL,
    Owner VARCHAR(255) NOT NULL,
    Description TEXT,
    URL VARCHAR(255) NOT NULL,
    Programming_Language VARCHAR(50),
    Creation_Date DATE,
    Last_Updated_Date DATE,
    Number_of_Stars INT,
    Number_of_Forks INT,
    Number_of_Open_Issues INT,
    License_Type VARCHAR(100)
);
"""
mycursor.execute(create_table_query)

# Insert DataFrame into MySQL table
insert_query = """
    INSERT INTO reposit (
        Repository_Name, 
        Owner, 
        Description, 
        URL, 
        Programming_Language, 
        Creation_Date, 
        Last_Updated_Date, 
        Number_of_Stars, 
        Number_of_Forks, 
        Number_of_Open_Issues, 
        License_Type
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df.iterrows():
    val = (
        row['Repository_Name'], 
        row['Owner'], 
        row['Description'], 
        row['URL'], 
        row['Programming_Language'], 
        row['Creation_Date'], 
        row['Last_Updated_Date'], 
        row['Number_of_Stars'], 
        row['Number_of_Forks'], 
        row['Number_of_Open_Issues'], 
        row['License_Type']
    )
    
    try:
        mycursor.execute(insert_query, val)
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Commit the transaction
mydb.commit()
print("Data saved to MySQL database!")

Data saved to MySQL database!


In [30]:
# Example Analysis

# Top 5 prog lang by the number of repositories
top_languages = df['Programming_Language'].value_counts().head(5)
print("Top 5 Programming Languages:")
print(top_languages)

# Top 5 repositories with the most stars
top_starred_repos = df[['Repository_Name', 'Number_of_Stars']].sort_values(by='Number_of_Stars', ascending=False).head(5)
print("\nTop 5 Starred Repositories:")
print(top_starred_repos)

# Average number of stars by programming language
avg_stars_per_language = df.groupby('Programming_Language')['Number_of_Stars'].mean().sort_values(ascending=False)
print("\nAverage Stars per Language:")
print(avg_stars_per_language.head(5))

Top 5 Programming Languages:
Programming_Language
Python              89
Jupyter Notebook    30
C++                 19
JavaScript          15
TypeScript           9
Name: count, dtype: int64

Top 5 Starred Repositories:
            Repository_Name  Number_of_Stars
150          awesome-python           221506
151  project-based-learning           200665
152                  Python           191326
180              tensorflow           186018
153              tensorflow           186018

Average Stars per Language:
Programming_Language
Shell     86748.000000
Java      48414.000000
HTML      41309.500000
Python    36801.292135
C++       32452.105263
Name: Number_of_Stars, dtype: float64
